## 1. Install the twitter library in python

In [ ]:
!pip install tweepy

## 2. Install library for JSON

In [ ]:
!pip install simplejson

## 3. Install sentiment analysis library

In [ ]:
!pip install textblob

# Streaming tweets and perform some data analysis


### Setting up and running a streaming crawler

In [ ]:
import tweepy
import simplejson as json
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from textblob import TextBlob
 
#Complete with your keys 

consumer_key = '8LbgPJVK5AgR1vKcvCssNyKkp'
consumer_secret = 'PPrIa7y3xcK4aktfqqEUeQuk2K8BzwmYsXoBZkmCaVygqW3J9M'
access_token = '191181457-zejYmLuj85d5XOzRxNX2tFIncNvqIEwnhqmzVjCb'
access_secret = 'LfQ7P7I1e47as04eptD2er246ZmJdhkQiLol6ARkC6nTP'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 

class MyListener(StreamListener):
    
    def __init__(self, api=None):
        super(StreamListener, self).__init__()
        self.num_tweets = 0

    def on_data(self, data):
        try:
            with open('MyFile.json', 'a') as f:
                if json.loads(data).get('place'):
                    #if json.loads(data)['place']['country'] == 'United States':
                        f.write(data) # This will store the whole JSON data in the file, you can perform some JSON filters
                        twitter_text = json.loads(data)['text'] # You can also print your tweets here
                        print(twitter_text)
                        print()
                        self.num_tweets += 1 
                        # Just to limit the number of tweets collected to check the 
                        # program at the beginning, then increase the limit
                        if self.num_tweets < 250: 
                            return True
                        else:
                            return False
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

 
    def on_error(self, status):
        print('Error :', status.place)
        return False
    
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=["coronavirus"], languages=['en']) # Add your keywords and other filters

print('_______ End _______')



### Store the JSON data in a CSV for analysing

In [ ]:
import simplejson as json

# Create the CSV file
with open ("MyFile.csv", 'w') as csv:
    # Write the title of the columns (features) that you want to store in the CSV file
    csv.write("id," + "created_at," + "text," + "country," +"country_code," + "followers," + "friends" + "\n")
    
    # Copy the data from the JSON file
    with open('MyFile.json', 'r') as jsonfile:
        for tweet in jsonfile: 
            data = json.loads(tweet)
            
            # The int values should be converted to strings
            csv.write(str(data["id"]) + ",")
            csv.write(str(data["created_at"]) + ",")
            csv.write((str(data["text"]).replace("\n","").replace(",","")) + ",")
            csv.write(str(data["place"]["country"]) + ",")
            csv.write(str(data["place"]["country_code"]) + ",")
            csv.write(str(data["user"]["followers_count"]) + ",")
            csv.write(str(data["user"]["friends_count"]))
            csv.write("\n")
            

### Load the previous CSV into pandas

In [ ]:
import pandas as pd
tweets = pd.read_csv('MyFile.csv', index_col=0, encoding='ISO-8859-1')
tweets.head(10)

### Analysing the polarity of the tweets

In [ ]:
from textblob import TextBlob
polarity, subjectivity = 0, 0

count = 0
for text in tweets.text:
    analysis = TextBlob(text)
    polarity += analysis.sentiment[0]
    subjectivity += analysis.sentiment[1]
    count +=1
    
print ("Average Polarity:", polarity/count)
print ("Average Subjectivity:",subjectivity/count)

### Generating a wordcloud

In [ ]:
!pip install wordcloud

In [ ]:
!pip install plotly
!pip install regexp

### Your own analysis

In [ ]:
tweets = pd.read_csv('MyFile.csv')

In [ ]:
# tweets.drop('Unnamed: 0', axis=1, inplace=True)
# tweets.drop('Title', axis=1, inplace=True)
tweets = tweets[~tweets['text'].isnull()]

def preprocess(text):
    text = text.str.replace("(<br/>)", "")
    text = text.str.replace('(<a).*(>).*(</a>)', '')
    text = text.str.replace('(&amp)', '')
    text = text.str.replace('(&gt)', '')
    text = text.str.replace('(&lt)', '')
    text = text.str.replace('(\xa0)', ' ')
    text = text.str.replace("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "")
    return text

tweets['text'] = preprocess(tweets['text'])

tweets['polarity'] = tweets['text'].map(lambda text: TextBlob(text).sentiment.polarity)
tweets['review_len'] = tweets['text'].astype(str).apply(len)
tweets['word_count'] = tweets['text'].apply(lambda x: len(str(x).split()))

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import re

text = tweets.text.values

STOPWORDS = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

wordcloud = WordCloud(
    width = 3000,
    height = 2000,
    background_color = 'white',
    stopwords = STOPWORDS).generate(str(text))
fig = plt.figure(
    figsize = (30, 20),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'gaussian')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
print('TOP 5 tweets with Positive (= +1) sentiment polarity: \n')

try:
    cl = tweets.loc[tweets.polarity == 1, ['text']].sample(5).values
    for c in cl:
        print(c[0])
except:
    print("[Error: not enought tweets to create a TOP 5] \n \n")
    print("[Showing the most positive tweets]: \n\n")
    
    cl = tweets.loc[tweets.polarity == 1, ['text']].values
    for c in cl:
        print(c[0],"\n")

In [ ]:
print('TOP 5 tweets with Neutral (=0) sentiment polarity: \n')

try:
    cl = tweets.loc[tweets.polarity == 0, ['text']].sample(5).values
    for c in cl:
        print(c[0])
except:
    print("[Error: not enought tweets to create a TOP 5] \n \n")
    print("[Showing the most positive tweets]: \n\n")
    
    cl = tweets.loc[tweets.polarity == 0, ['text']].values
    for c in cl:
        print(c[0],"\n")

In [ ]:
print('TOP 5 tweets with Negative (= -1) sentiment polarity: \n')

try:
    cl = tweets.loc[tweets.polarity == -1, ['text']].sample(5).values
    for c in cl:
        print(c[0])
except:
    print("[Error: not enought tweets to create a TOP 5] \n \n")
    cl = tweets.loc[tweets.polarity == -1, ['text']].values
    
    if cl.size > 0:
        print("[Showing the most negative tweets:] \n\n")
    
        cl = tweets.loc[tweets.polarity ==-1, ['text']].values
        for c in cl:
            print(c[0],"\n")
        
    else:
        print ("[Error: Could not find any tweet with polarity equal to -1]\n")
        print ("[printing some of the most negative tweeets (<0)]\n\n")
        
        cl = tweets.loc[tweets.polarity < 0, ['text']].sample(5).values
        for c in cl:
            print(c[0],"\n")

In [ ]:
import plotly.express as px

fig = px.histogram(tweets, x="country")
fig.show()

In [ ]:
i=0
tweets_country_1, tweets_country_2, tweets_country_3= [], [], []
polarity_c1, polarity_c2, polarity_c3= [], [], []


while i < len(tweets):
    if tweets["country"][i] == "United States":
        tweets_country_1.append(tweets["text"][i])
    elif tweets["country"][i] == "United Kingdom":
        tweets_country_2.append(tweets["text"][i])
    elif tweets["country"][i] == "Malaysia":
        tweets_country_3.append(tweets["text"][i])
    i += 1


count = 0
for text in tweets_country_1:
    analysis = TextBlob(text)
    polarity_c1.append(analysis.sentiment[0])
for text in tweets_country_2:
    analysis = TextBlob(text)
    polarity_c2.append(analysis.sentiment[0])
for text in tweets_country_3:
    analysis = TextBlob(text)
    polarity_c3.append(analysis.sentiment[0])

In [ ]:
import plotly.graph_objects as go
import numpy as np

x0 = polarity_c1
x1 = polarity_c2 
x2 = polarity_c3

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name="USA"))
fig.add_trace(go.Histogram(x=x1, name="UK"))
fig.add_trace(go.Histogram(x=x2, name="Malaysia"))

# Overlay both histograms
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.65)
fig.show()

In [ ]:
countries = tweets["country"]
countries = list(dict.fromkeys(countries))
polarity_per_country, tweets_country, polarities= [], [], []

for c in countries: 
    i=0
    while i < len(tweets):
        if tweets["country"][i] == c:
            tweets_country.append(tweets["text"][i])
        i += 1

    count = 0
    for text in tweets_country:
        analysis = TextBlob(text)
        polarity += analysis.sentiment[0]
        count +=1
    polarity = polarity/count
    polarities.append(polarity)
    


In [ ]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure(data=go.Choropleth(
    locationmode= "country names",
    locations = countries,
    z = polarities,
    text = countries,
    colorscale = 'rainbow',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'polarities',
))

fig.update_layout(
    title_text='Sentiment Analysis',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: Twitter Data, Hashtag #coronavirus',
        showarrow = False
    )]
)

fig.show()